## explore the output chanks

In [1]:
import os, glob
import xarray as xr
import rioxarray
from rioxarray.merge import merge_datasets, merge_arrays
import numpy as np
import geopandas as gpd
import pandas as pd
from equi7grid.equi7grid import Equi7Grid

/home/return-mmilenkovic/miniconda3/envs/yeoda/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [ ]:
# chek the avaliable memory
!free -h

In [ ]:
# folder wih the output files:
data_folder = r'/project/return/Share/mm/S1_SA_TEST_UPSCALE/TILE_WISE/AOI_PA/E078N066T3'
#
#myOrbit = 'D141'
myOrbit = 'D039'
#
myPol = 'VH'


In [ ]:
# get all nc-files in a list
ncFileList = glob.glob(data_folder + '/**/*' + myOrbit + '_' + myPol + '.nc', recursive=True)

In [ ]:
# sorted list:
ncFileList = sorted(ncFileList)

In [ ]:
ncRows = [os.path.basename(ncPath).split('_')[1] for ncPath in ncFileList]
ncCols = [os.path.basename(ncPath).split('_')[2] for ncPath in ncFileList]

In [ ]:
df_ncFiles = pd.DataFrame(list(zip(ncRows, ncCols, ncFileList)), columns =['row', 'col', 'path'])

In [ ]:
df_ncFiles = df_ncFiles.astype({'row': 'uint8', 'col': 'uint8'})

In [ ]:
df_ncFiles = df_ncFiles.sort_values(['row', 'col'])

In [ ]:
# ignore warnings about missing spatial refernce in the nc files:
import warnings
import rasterio
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

In [ ]:
# Equi7 SA wkt:
PROJ = 'PROJCS["Azimuthal_Equidistant",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.017453292519943295]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["false_easting",7257179.23559],PARAMETER["false_northing",5592024.44605],PARAMETER["central_meridian",-60.5],PARAMETER["latitude_of_origin",-14.0],UNIT["Meter",1.0]]'

In [ ]:
ds = rioxarray.open_rasterio(df_ncFiles.path[0])

In [ ]:
ds

In [ ]:
# specify variables to load
deforSet = ['mean_2019', 'exception_label', 'max_mag', 'max_mag_date', 't_pre']
regrowthSet2017 = ['mean_2017', 'median_2017', 'std_2017', 'MAD_2017', 'q90_2017', 'q10_2017', 'q75_2017', 'q25_2017', 'max_2017', 'min_2017']
#ds = rioxarray.open_rasterio(df_ncFiles.path[0], variable=varSet1)

In [ ]:
# read multiple ckunks:
ChunkDatasetList = []
# specify what rows and colomns to read
myRows, myCols  = np.meshgrid(np.arange(41, 49), np.arange(0, 60))
#
for myRow, myCol in zip(myRows.flatten(), myCols.flatten()):
    # set filename:
    currentItem = df_ncFiles[(df_ncFiles['row'] == myRow) & (df_ncFiles['col'] == myCol)]
    if currentItem.empty:
        continue
    #    
    fileName = currentItem.path.values[0]
    # read the chunk
    aux_ds = rioxarray.open_rasterio(fileName, variable=deforSet)
    # set crs:
    aux_ds.rio.write_crs(PROJ, inplace=True)
    # remove band coordinate:
    aux_ds.squeeze('band').drop('band')
    # append chank list:
    ChunkDatasetList.append(aux_ds)      

In [ ]:
# mearge chanks spatialy:
my_out = merge_datasets(ChunkDatasetList)   

In [ ]:
# correct corrdinates ('open_rasterio' treats coordinates as pixel centers)
pixSize = 20
my_out = my_out.assign_coords(x=my_out.x +  pixSize/2)
my_out = my_out.assign_coords(y=my_out.y -  pixSize/2)

In [ ]:
my_out.nbytes/1e6

In [ ]:
my_out.mean_2019.rio.to_raster("test_mean2019.tif")

In [ ]:
#%matplotlib widget
%matplotlib inline
ax = my_out.mean_2019.plot(robust=True, cmap='gray')
ax.axes.set_aspect('equal')

In [ ]:
# select only alerts with high magnitude:
defor_ds = my_out.where((my_out.exception_label==0) & (my_out.max_mag>3))
# calculate the deforestation date (the t_pre is converted from days to parth of the year as the max_mag_date):
defor_ds['deforDate'] = defor_ds.max_mag_date - np.abs(defor_ds.t_pre/365.)

In [ ]:
#%matplotlib widget
%matplotlib inline
ax =defor_ds.deforDate.plot(figsize=(10, 8), levels=[2017, 2018, 2019, 2020, 2021])
ax.axes.set_aspect('equal')

In [ ]:
defor_ds.deforDate.rio.to_raster("test_deforDate.tif")

# Deal with Equi7Grid:

In [ ]:
e7_20 = Equi7Grid(20)

In [ ]:
e7_20.SA.tilesys.decode_tilename('E078N066T3')

In [ ]:
e7_myTile = e7_20.SA.tilesys.create_tile('E078N066T3')

In [ ]:
e7_myTile.ij2xy(0.5, 0.5)

In [ ]:
e7_myTile.ij2xy(1, 1, offset='center')

In [ ]:
aa = [e7_myTile.ij2xy(myX, myY) for myX, myY in zip(ds.x.values, ds.y.values)]

In [ ]:
newX, newY = list(zip(*aa))

In [ ]:
ds

In [ ]:
ds = ds.assign_coords(x=np.array(newX))
ds = ds.assign_coords(y=np.flip(np.array(newY)))

In [ ]:
ds.max_mag.plot()

In [ ]:
type(e7_myTile.ij2xy(0, 0))